In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from importlib import reload
from time import time
import pickle
from copy import deepcopy
import re


import numpy as np
import scipy as sc
import pandas as pd
import torch
print(torch.cuda.is_available())
from statsmodels.stats.weightstats import _zstat_generic

import models
import dre
import rna
from calibrate import rectified_p_value
import vae
import designers
import editdistance

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["font.size"] = 12

import seaborn as sns
sns.set_style('whitegrid')

True


2024-05-06 00:22:25.642346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 00:22:25.642381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 00:22:25.643490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 00:22:25.650149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 00:22:27.407990: W tensorflow/comp

In [22]:
# load calibration and training data
noise_sd = 0.02
p_mut = 4 / 50
model_and_data_path = '/homefs/home/wongfanc/density-ratio-estimation/rna-models'
model_and_data_fname_no_ftype = 'nsd0.02-040324'
n_hidden = 100
n_filters = 32

cal_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-results/calibration-pm0.08-nsd0.02-040524.npz'
d = np.load(cal_fname)
calseqs_n = list(d['calseqs_n'])
ycal_n = d['ycal_n']
print(f'Loaded {ycal_n.size} training points from {cal_fname}.\n')

train_fname = os.path.join(model_and_data_path, 'traindata-' + model_and_data_fname_no_ftype + '.npz')
d = np.load(train_fname)
trainseq_n = list(d['trainseq_n'])
ytrain_n = d['ytrain_n']
print(f'Loaded {ytrain_n.size} training points from {train_fname}.\n')

Loaded 5000 training points from /homefs/home/wongfanc/density-ratio-estimation/rna-results/calibration-pm0.08-nsd0.02-040524.npz.

Loaded 5000 training points from /homefs/home/wongfanc/density-ratio-estimation/rna-models/traindata-nsd0.02-040324.npz.



In [5]:
# load labeled design sequences
# (don't need viennaenv from this point on)
reload(dre)

design_pkl_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-results/designs-041124-copy.pkl'
with open(design_pkl_fname, 'rb') as f:
    name2designdata = pickle.load(f)
    
# prepare intermediate iterations for C/DbAS
train_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-models/traindata-nsd0.02-040324.npz'
design_pkl_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-results/designs-041124-copy.pkl'
name2designdata = dre.prepare_name2designdata(design_pkl_fname, train_fname)

Loaded 5000 training points from /homefs/home/wongfanc/density-ratio-estimation/rna-models/traindata-nsd0.02-040324.npz.

Removed dbas-ridge-0.6t0 designs.
Removed dbas-ridge-0.6t1 designs.
Removed dbas-ridge-0.6t2 designs.
Removed dbas-ridge-0.6t3 designs.
Removed dbas-ridge-0.6t4 designs.
Removed dbas-ridge-0.6t5 designs.
Removed dbas-ridge-0.6t6 designs.
Removed dbas-ridge-0.6t7 designs.
Removed dbas-ridge-0.6t8 designs.
Removed dbas-ridge-0.6t9 designs.
Removed dbas-ridge-0.6t10 designs.
Removed dbas-ridge-0.6t11 designs.
Removed dbas-ridge-0.6t12 designs.
Removed dbas-ridge-0.6t13 designs.
Removed dbas-ridge-0.6t14 designs.
Removed dbas-ridge-0.6t15 designs.
Removed dbas-ridge-0.6t16 designs.
Removed dbas-ridge-0.6t17 designs.
Removed dbas-ridge-0.6t18 designs.
Removed dbas-ridge-0.6t19 designs.
Removed dbas-ridge-0.7t0 designs.
Removed dbas-ridge-0.7t1 designs.
Removed dbas-ridge-0.7t2 designs.
Removed dbas-ridge-0.7t3 designs.
Removed dbas-ridge-0.7t4 designs.
Removed dbas-ridge

  Removed cbas-ridge-0.3t17
  Removed cbas-ridge-0.3t18
Iterations [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] (interval 1) have mean consecutive diff in mean prediction of 0.014.
Iterations [1, 3, 5, 7, 9, 11, 13, 15, 17, 19] (interval 2) have mean consecutive diff in mean prediction of 0.028.
Iterations [2, 5, 8, 11, 14, 17, 19] (interval 3) have mean consecutive diff in mean prediction of 0.040.
Iterations [3, 7, 11, 15, 19] (interval 4) have mean consecutive diff in mean prediction of 0.057.
Iterations [4, 9, 14, 19] (interval 5) have mean consecutive diff in mean prediction of 0.077.
Iterations [5, 11, 17, 19] (interval 6) have mean consecutive diff in mean prediction of 0.077.
Iterations [6, 13, 19] (interval 7) have mean consecutive diff in mean prediction of 0.112.
Using the following iterations for dbas-ridge-0.3: [6, 13, 19].
  Removed dbas-ridge-0.3t0
  Removed dbas-ridge-0.3t1
  Removed dbas-ridge-0.3t2
  Removed dbas-ridge-0.3t3
  Removed dbas-ri

In [7]:
# check MultiMDRE runs smoothly on all design algorithms
reload(dre)
n_epoch = 500
group_regex_strs = [
    'adalead-ridge-0.\d',
    'adalead-ff-0.\d',
    'adalead-cnn-0.\d',
    'biswas-ridge-0.\d',
    'biswas-ff-0.\d',
    'biswas-cnn-0.\d',
    'dbas-ff-0.\d',
    'dbas-cnn-0.\d',
    'pex-ridge',
    'pex-ff',
    'pex-cnn',
    'dbas-ridge-0.1t\d',
    'dbas-ridge-0.2t\d',
    'dbas-ridge-0.3t\d',
    'dbas-ridge-0.4t\d',
    'dbas-ridge-0.5t\d',
    'cbas-ridge-0.1t\d',
    'cbas-ridge-0.2t\d',
    'cbas-ridge-0.3t\d',
    'cbas-ridge-0.4t\d',
    'cbas-ridge-0.5t\d',
    'cbas-ridge-0.6t\d',
    'cbas-ridge-0.7t\d',
    'cbas-ridge-0.8t\d',
    'cbas-ridge-0.9t\d'
]
device = torch.device('cuda')

mdre = dre.MultiMDRE(
    group_regex_strs,
    device=device
)
mdre.fit(
    name2designdata,
    n_epoch=n_epoch,
    verbose=True
)

Fitting MDRE for adalead-ridge-0.\d, which has 7 design algorithms:
train
adalead-ridge-0.5
adalead-ridge-0.4
adalead-ridge-0.3
adalead-ridge-0.2
adalead-ridge-0.1
adalead-ridge-0.05
adalead-ridge-0.01
One-hot-encoding all 8 categories of sequences...
  Done. (4 s)


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 33.77it/s]


Min train loss 0.84, min val loss 0.77

Fitting MDRE for adalead-ff-0.\d, which has 7 design algorithms:
train
adalead-ff-0.5
adalead-ff-0.4
adalead-ff-0.3
adalead-ff-0.2
adalead-ff-0.1
adalead-ff-0.05
adalead-ff-0.01
One-hot-encoding all 8 categories of sequences...
  Done. (4 s)


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 34.13it/s]


Min train loss 0.82, min val loss 0.77

Fitting MDRE for adalead-cnn-0.\d, which has 7 design algorithms:
train
adalead-cnn-0.5
adalead-cnn-0.4
adalead-cnn-0.3
adalead-cnn-0.2
adalead-cnn-0.1
adalead-cnn-0.05
adalead-cnn-0.01
One-hot-encoding all 8 categories of sequences...
  Done. (4 s)


...

Min train loss 0.18, min val loss 0.39

Fitting MDRE for cbas-ridge-0.8t\d, which has 1 design algorithms:
train
cbas-ridge-0.8t19
One-hot-encoding all 2 categories of sequences...
  Done. (0 s)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 184.79it/s]


Min train loss 0.08, min val loss 0.24

Fitting MDRE for cbas-ridge-0.9t\d, which has 1 design algorithms:
train
cbas-ridge-0.9t19
One-hot-encoding all 2 categories of sequences...
  Done. (0 s)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 186.38it/s]

Min train loss 0.00, min val loss 0.03



In [6]:
# run selection experiments!
reload(dre)
reload(rna)

# ----- build menu of design algorithm names -----
adalead_thresholds = [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.01]
biswas_temperatures = [0.02, 0.015, 0.01, 0.005]
cbas_dbas_quantiles = np.arange(0.1, 0.91, 0.1)

design_names = []
for q in cbas_dbas_quantiles:
    q = round(q, 2)
    design_names.append(f'cbas-ridge-{q}t19')
    
for q in cbas_dbas_quantiles[: 2]:
    q = round(q, 2)
    design_names.append(f'dbas-ridge-{q}t19')
    
for model_name in ['ridge', 'ff', 'cnn']:
    
    
    for thresh in adalead_thresholds:
        thresh = round(thresh, 4)
        design_names.append(f'adalead-{model_name}-{thresh}')
        
    for temp in biswas_temperatures:
        temp = round(temp, 4)
        design_names.append(f'biswas-{model_name}-{temp}') 
    
    design_names.append(f'pex-{model_name}') 
        
    if model_name != 'ridge':
        for q in cbas_dbas_quantiles:
            q = round(q, 2)
            design_names.append(f'dbas-{model_name}-{q}')
            
print(f'Testing {len(design_names)} design names:')
for name in design_names:
    print(f'  {name}')
    
# 'adalead-ridge-0.[0-9]5$',
# 'adalead-ridge-0.[0-9][1]?$',
# 'adalead-ff-0.[0-9]5$',
# 'adalead-ff-0.[0-9][1]?$',
# 'adalead-cnn-0.[0-9]5$',
# 'adalead-cnn-0.[0-9][1]?$',

mdre_group_regex_strs = [
    'adalead-ridge-0.\d',
    'adalead-ff-0.\d',
    'adalead-cnn-0.\d',
    'biswas-ridge-0.\d',
    'biswas-ff-0.\d',
    'biswas-cnn-0.\d',
    'dbas-ff-0.\d',
    'dbas-cnn-0.\d',
    'pex-ridge',
    'pex-ff',
    'pex-cnn',
    'dbas-ridge-0.1t\d',
    'dbas-ridge-0.2t\d',
#     'dbas-ridge-0.3t\d',
    # 'dbas-ridge-0.4t\d',
    # 'dbas-ridge-0.5t\d',
    # 'dbas-ridge-0.6t\d',
    'cbas-ridge-0.1t\d',
    'cbas-ridge-0.2t\d',
    'cbas-ridge-0.3t\d',
    'cbas-ridge-0.4t\d',
    'cbas-ridge-0.5t\d',
    'cbas-ridge-0.6t\d',
    'cbas-ridge-0.7t\d',
    'cbas-ridge-0.8t\d',
    'cbas-ridge-0.9t\d'
]
    

# where to save results!
results_csv_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv'
design_pkl_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-results/designs-041124-copy.pkl'
model_and_data_fname_no_ftype = 'nsd0.02-040324'
cal_pkl_fname = '/homefs/home/wongfanc/density-ratio-estimation/rna-data/calibration-pm0.08-nsd0.02_041524_2.pkl'
target_values = np.arange(0.25, 0.501, 0.001)
n_trial = 200
n_mdre_hidden = 500
n_mdre_epoch = 100
device = torch.device('cuda')

df = rna.run_selection_experiments(
    design_names,
    design_pkl_fname,
    model_and_data_fname_no_ftype,
    cal_pkl_fname,
    mdre_group_regex_strs,
    target_values,
    n_trial,
    n_mdre_hidden=n_mdre_hidden,
    n_mdre_epoch=n_mdre_epoch,
    results_csv_fname=results_csv_fname,
    device=device,
)

Testing 77 design names:
  cbas-ridge-0.1t19
  cbas-ridge-0.2t19
  cbas-ridge-0.3t19
  cbas-ridge-0.4t19
  cbas-ridge-0.5t19
  cbas-ridge-0.6t19
  cbas-ridge-0.7t19
  cbas-ridge-0.8t19
  cbas-ridge-0.9t19
  dbas-ridge-0.1t19
  dbas-ridge-0.2t19
  adalead-ridge-0.5
  adalead-ridge-0.45
  adalead-ridge-0.4
  adalead-ridge-0.35
  adalead-ridge-0.3
  adalead-ridge-0.25
  adalead-ridge-0.2
  adalead-ridge-0.15
  adalead-ridge-0.1
  adalead-ridge-0.05
  adalead-ridge-0.01
  biswas-ridge-0.02
  biswas-ridge-0.015
  biswas-ridge-0.01
  biswas-ridge-0.005
  pex-ridge
  adalead-ff-0.5
  adalead-ff-0.45
  adalead-ff-0.4
  adalead-ff-0.35
  adalead-ff-0.3
  adalead-ff-0.25
  adalead-ff-0.2
  adalead-ff-0.15
  adalead-ff-0.1
  adalead-ff-0.05
  adalead-ff-0.01
  biswas-ff-0.02
  biswas-ff-0.015
  biswas-ff-0.01
  biswas-ff-0.005
  pex-ff
  dbas-ff-0.1
  dbas-ff-0.2
  dbas-ff-0.3
  dbas-ff-0.4
  dbas-ff-0.5
  dbas-ff-0.6
  dbas-ff-0.7
  dbas-ff-0.8
  dbas-ff-0.9
  adalead-cnn-0.5
  adalead-cnn-0.45


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.20it/s]


Min train loss 1.57, min val loss 1.54

Fitting MDRE for adalead-ff-0.\d, which has 11 design algorithms:
train
adalead-ff-0.5
adalead-ff-0.4
adalead-ff-0.3
adalead-ff-0.2
adalead-ff-0.1
adalead-ff-0.05
adalead-ff-0.01
adalead-ff-0.45
adalead-ff-0.35
adalead-ff-0.25
adalead-ff-0.15
One-hot-encoding all 12 categories of sequences...
  Done. (8 s)


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.21it/s]


Min train loss 1.52, min val loss 1.48

Fitting MDRE for adalead-cnn-0.\d, which has 11 design algorithms:
train
adalead-cnn-0.5
adalead-cnn-0.4
adalead-cnn-0.3
adalead-cnn-0.2
adalead-cnn-0.1
adalead-cnn-0.05
adalead-cnn-0.01
adalead-cnn-0.45
adalead-cnn-0.35
adalead-cnn-0.25
adalead-cnn-0.15
One-hot-encoding all 12 categories of sequences...
  Done. (8 s)


...

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 93.66it/s]


Min train loss 0.02, min val loss 0.02

Done running 1 / 200 trials (140 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (141 s).

Done running 2 / 200 trials (280 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (281 s).

Done running 3 / 200 trials (421 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (422 s).

Done running 4 / 200 trials (561 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (561 s).

Done running 5 / 200 trials (701 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (702 s).

Done running 6 / 200 trials (841 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (842 s).

Done running 7 / 200 trials (981 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.c

Done running 59 / 200 trials (8282 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8284 s).

Done running 60 / 200 trials (8424 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8426 s).

Done running 61 / 200 trials (8565 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8567 s).

Done running 62 / 200 trials (8706 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8708 s).

Done running 63 / 200 trials (8847 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8849 s).

Done running 64 / 200 trials (8988 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (8990 s).

Done running 65 / 200 trials (9129 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (9131 s).

Done r

Done running 116 / 200 trials (16339 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (16342 s).

Done running 117 / 200 trials (16481 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (16483 s).

Done running 118 / 200 trials (16622 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (16625 s).

Done running 119 / 200 trials (16764 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (16767 s).

Done running 120 / 200 trials (16906 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (16909 s).

Done running 121 / 200 trials (17048 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (17050 s).

Done running 122 / 200 trials (17189 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.c

Done running 173 / 200 trials (24448 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (24452 s).

Done running 174 / 200 trials (24591 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (24595 s).

Done running 175 / 200 trials (24734 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (24738 s).

Done running 176 / 200 trials (24877 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (24881 s).

Done running 177 / 200 trials (25019 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (25023 s).

Done running 178 / 200 trials (25163 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.csv (25167 s).

Done running 179 / 200 trials (25305 s).
Saved to /homefs/home/wongfanc/density-ratio-estimation/rna-results/selection-v0-050524.c

In [ ]:
# generate, label, and save calibration data for selection experiments
# (need viennaenv)

n_trial = 800
n_cal = 5000
noise_sd = 0.02
p_mutation = 4 / 50
data_path = '/homefs/home/wongfanc/density-ratio-estimation/rna-data/'
fname = os.path.join(data_path, 'calibration-pm{:.2f}-nsd{:.2f}_041524_2.pkl'.format(p_mutation, noise_sd))

landscape = rna.RNABinding(noise_sd=noise_sd)
calibration_data = []
t0 = time()
for t in range(n_trial):
    calseqs_n, ycal_n = landscape.get_training_data(n_cal, p_mutation)
    calibration_data.append((calseqs_n, ycal_n))
    with open(fname, 'wb') as f:
        pickle.dump(calibration_data, f)
    print(f'trial {t + 1} / {n_trial} ({int(time() - t0)} s)\n')